In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib.colors import LogNorm
from matplotlib import rc
from numpy import inf
from matplotlib import gridspec

import os

rc('text', usetex=True)

import matplotlib as mpl
rc('font', family='serif')
rc('text', usetex=True)
rc('font', size=22)
rc('xtick', labelsize=15)
rc('ytick', labelsize=15)
rc('legend', fontsize=15)

#
mpl.rcParams.update({'font.size': 19})
#mpl.rcParams.update({'legend.fontsize': 18})
mpl.rcParams.update({'xtick.labelsize': 18}) 
mpl.rcParams.update({'ytick.labelsize': 18}) 
mpl.rcParams.update({'text.usetex' : False})
mpl.rcParams.update({'axes.labelsize': 18}) 
mpl.rcParams.update({'legend.frameon': False}) 

import matplotlib.pyplot as plt
import mplhep as hep
#hep.set_style(hep.style.CMS)
hep.set_style("CMS") 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/tmp/ipykernel_14228/1641985075.py:42: FutureWarning: ``set_style`` is deprecated: Naming convention is changing to match mpl. Use ``mplhep.style.use()``.
  hep.set_style("CMS")


In [4]:
data = pd.read_pickle("/clusterfs/ml4hep/yxu2/unfolding_mc_inputs/Data_nominal.pkl")
mc = pd.read_pickle("/clusterfs/ml4hep/yxu2/unfolding_mc_inputs/Rapgap_nominal.pkl")

In [5]:
theta_unknown_S = data[['e_px','e_py','e_pz','jet_pt','jet_eta','jet_phi','jet_dphi','jet_qtnorm']].to_numpy()
theta0_S = mc[['e_px','e_py','e_pz','jet_pt','jet_eta','jet_phi','jet_dphi','jet_qtnorm']].to_numpy()
theta0_G = mc[['gene_px','gene_py','gene_pz','genjet_pt','genjet_eta','genjet_phi','genjet_dphi','genjet_qtnorm']].to_numpy()
weights_MC_sim = mc['wgt']
pass_reco = np.array(mc['pass_reco'])
pass_truth = np.array(mc['pass_truth'])
pass_fiducial = np.array(mc['pass_fiducial'])

In [9]:
#Data Unfolded From Rapgap
cuts_h1rpgp       = np.load('from_rapgap_cuts.npy')
jet_pT_h1rpgp     = np.load('from_rapgap_jet_pT.npy')[cuts_h1rpgp]
q_perp_h1rpgp     = np.load('from_rapgap_q_perp.npy')[cuts_h1rpgp]
asymm_phi_h1rpgp  = np.load('from_rapgap_asymm_angle.npy')[cuts_h1rpgp]
weights_h1rpgp    = np.load('from_rapgap_weights.npy')[cuts_h1rpgp]
mc_weights_h1rpgp = np.load("from_rapgap_mc_weights.npy")[cuts_h1rpgp]
nn_weights_h1rpgp = np.load("from_rapgap_nn_weights.npy")[cuts_h1rpgp]

In [6]:
symbol = {}
symbol['dphi'] = '$\Delta\phi$ [rad]'
symbol['jeteta'] = '$\eta^{jet}$'
symbol['qt']  = '$q_{T}/Q$'
symbol['jetpt'] = '$p_{T}^{jet}$ [GeV]'
mylabels = ['jetpt','jeteta','dphi','qt']

mymap = {}
mymap[0] = 3
mymap[1] = 4
mymap[2] = 6
mymap[3] = 7

In [7]:
bins = {}
dologx = [False,False,False,True]
dology = [True,False,True,True]

#jet pt
bins[0] = np.logspace(np.log10(10),np.log10(100),7)

#jet eta
bins[1] = np.linspace(-1,2.5,6)

#dphi
bins[2] = np.logspace(np.log10(0.03),np.log10(np.pi/2.0),9) - 0.03
bins[2] = bins[2][1:]
bins[2][0] = 0.0

#qt
bins[3] = np.logspace(np.log10(0.03),np.log10(3.03),9) - 0.03
bins[3] = bins[3][1:]
bins[3][0] = 0.0

In [18]:
unfolded_vals=np.stack((q_perp_h1rpgp,asymm_phi_h1rpgp))
do_it =True
for jj in range(1):

    cov_mat = np.zeros([len(bins[jj])-1,len(bins[jj])-1])
    for i in range(len(bins[jj])-1):
            for j in range(len(bins[jj])-1):
                #Stat. uncert
                EX = np.mean(np.array([unfolded_vals[ii,jj] for ii in range(1,100)])[:,i])
                EY = np.mean(np.array([unfolded_vals[ii,jj] for ii in range(1,100)])[:,j])
                EXY = np.mean(np.array(np.array([unfolded_vals[ii,jj] for ii in range(1,100)])[:,i]*np.array([unfolded_vals[ii,jj] for ii in range(1,100)])[:,j]))
                cov_mat[i,j]+=EXY-EX*EY
            
    cov_mat = np.array(cov_mat)
    cor_mat = cov_mat.copy()
    for i in range(len(bins[jj])-1):
        for j in range(len(bins[jj])-1):
            cor_mat[i,j]=cov_mat[i,j]/(cov_mat[i,i]*cov_mat[j,j])**0.5

    fig = plt.figure(figsize=(7, 5)) 
    gs = gridspec.GridSpec(1, 1, height_ratios=[1]) 
    ax0 = plt.subplot(gs[0])
    ax0.yaxis.set_ticks_position('both')
    ax0.xaxis.set_ticks_position('both')
    ax0.tick_params(direction="in",which="both")
    ax0.minorticks_on()
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)

    plt.imshow(cor_mat,vmin = -1,vmax = 1,origin='lower',cmap="RdBu_r")
    cbar = plt.colorbar()
    cbar.ax.set_ylabel('Correlation') 
    
    plt.text(0.25, 1.3,'H1',
                  horizontalalignment='center',
         verticalalignment='center',
         transform = ax0.transAxes, fontsize=25, fontweight='bold')
    plt.text(0.25, 1.2,'Stat. Uncertainty',
                  horizontalalignment='center',
         verticalalignment='center',
         transform = ax0.transAxes, fontsize=20)
    plt.text(0.8, 1.3,'$Q^{2}>$150 GeV$^{2}$ \n $0.2<y<0.7$ \n $p_{T}^{jet}>10$ GeV  \n $k_{T}, R=1.0$',
         horizontalalignment='center',
         verticalalignment='center',
         transform = ax0.transAxes, fontsize=20)   

    plt.ylabel(symbol[mylabels[jj]])
    plt.xlabel(symbol[mylabels[jj]])
    mylabs = [""]
    if (j<2):
        for i in range(len(bins[jj])-1):
            mylabs+=["%0.0f" % (0.5*(bins[jj][i]+bins[jj][i+1]))]
    else:
        for i in range(len(bins[jj])-1):
            mylabs+=["%0.1f" % (0.5*(bins[jj][i]+bins[jj][i+1]))]
    ax0.set_xticklabels(mylabs)
    ax0.set_yticklabels(mylabs)
    ax0.xaxis.set_major_locator(plt.MaxNLocator(len(bins[jj])))
    ax0.yaxis.set_major_locator(plt.MaxNLocator(len(bins[jj])))
        
    fig.savefig('figures/CorStatMat_%s.pdf'%mylabels[jj],bbox_inches='tight')

IndexError: index 2 is out of bounds for axis 0 with size 2